In [1]:
import pandas as pd
import scipy.stats as st
from datetime import datetime
from datetime import timedelta
import math
import numpy as np
from scipy import stats
import copy
from PIL import Image
# import imageio

In [2]:
class Imageee:
    def generate_imageee(path, array):
        print("generate_imageee method in...")
        print("이미지 저장 전 구조: {}".format(array.shape))
        
#         array = img.astype(np.uint8)
#         imageio.imwrite(path, array)
        img = Image.fromarray(array.astype('uint8'), 'L')
        img.save(path)
        
    def propagate_2(array1, array2):
        for row in range(array1.shape[0]):
            for col in range(array1.shape[1]):
                array2[row][col] += array1[row][col]
        return array2

In [3]:
class Route:
    def __init__(self):
        self.init_route()
        self.init_club_route()
        self.init_dates()
        self.init_club_dates()
        self.init_causes()
        
    def init_route(self):
        patient_route = pd.read_csv('covid19/PatientRoute.csv')
        patient_info = pd.read_csv('covid19/PatientInfo.csv')
        patient_info = patient_info[["patient_id","global_num","sex","age","infection_case"]]

        merge_route = pd.merge(patient_info, patient_route)
        merge_route = merge_route[merge_route["province"]=="Seoul"]
        merge_route = merge_route[pd.notnull(merge_route['sex'])]
        merge_route = merge_route[pd.notnull(merge_route['age'])]
        
        self.route = merge_route
        
    def init_club_route(self):
        club_route = pd.read_csv('club.csv')
        club_route = club_route[pd.notnull(club_route['latitude'])]
        club_route = club_route[pd.notnull(club_route['longitude'])]
        club_route = club_route[club_route["date"]!="2020-04-20"]
        self.club_route = club_route
        
    def init_dates(self):
        route_dates = self.route['date']
        route_dates = route_dates.drop_duplicates(keep='last')
        route_dates = route_dates.tolist()
        route_dates.sort()
        self.route_dates = route_dates
        
    def init_club_dates(self):
        club_dates = self.club_route['date']
        club_dates = club_dates.drop_duplicates(keep='last')
        club_dates = club_dates.tolist()
        club_dates.sort()
        self.club_dates = club_dates
        
    def init_causes(self):
        causes = self.route['infection_case']
        causes = causes.drop_duplicates(keep='last')
        causes = causes.dropna()
        causes = causes.tolist()

        causes_stay = ["contact with patient", "overseas inflow", "etc"]
        for i, cause in enumerate(causes):
            if cause in causes_stay:
                continue
            causes[i] = "community infection"

        self.causes = list(dict.fromkeys(causes))
        
    def get_route_info(self):
        return self.route, self.route_dates
        
    def get_club_route_info(self):
        return self.club_route, self.club_dates

In [4]:
class GridMap:
    Re = 6371.00877  ##  지도반경
    grid = 0.16  ##  격자간격 (km)
    slat1 = 30.0  ##  표준위도 1
    slat2 = 60.0  ##  표준위도 2
    olon = 127.0175205  ##  기준점 경도
    olat = 37.523124  ##  기준점 위도
    xo = 18.4 / grid #  기준점 X좌표
    yo = 21.6 / grid  ##  기준점 Y좌표
    row = 256
    column = 256
    seoul_start_lat = 37.698098
    seoul_start_lon = 126.799791
    seoul_end_lat = 37.34815
    seoul_end_lon = 127.23525

    def __init__(self):
        self.PI = math.asin(1.0) * 2.0
        self.DEGRAD = self.PI / 180.0
        self.RADDEG = 180.0 / self.PI

        self.re = self.Re / self.grid
        self.slat1 = self.slat1 * self.DEGRAD
        self.slat2 = self.slat2 * self.DEGRAD
        self.olon = self.olon * self.DEGRAD
        self.olat = self.olat * self.DEGRAD

        self.sn = math.tan(self.PI * 0.25 + self.slat2 * 0.5) / \
                  math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sn = math.log(math.cos(self.slat1) /
                           math.cos(self.slat2)) / math.log(self.sn)
        self.sf = math.tan(self.PI * 0.25 + self.slat1 * 0.5)
        self.sf = math.pow(self.sf, self.sn) * math.cos(self.slat1) / self.sn
        self.ro = math.tan(self.PI * 0.25 + self.olat * 0.5)
        self.ro = self.re * self.sf / math.pow(self.ro, self.sn)

    def toGrid(self, lat, lon):
        lat = float(lat)
        lon = float(lon)
        ra = math.tan(self.PI * 0.25 + lat * self.DEGRAD * 0.5)
        ra = self.re * self.sf / pow(ra, self.sn)
        theta = lon * self.DEGRAD - self.olon
        if theta > self.PI:
            theta -= 2.0 * self.PI
        if theta < self.PI * -1:
            theta += 2.0 * self.PI
        theta *= self.sn
        x = (ra * math.sin(theta)) + self.xo
        y = (self.ro - ra * math.cos(theta)) + self.yo
        x = int(x + 1.5)
        # y = int(y + 1.5)
        y = (int(y + 1.5) - 255) * -1
        return x, y

    def isSeoul(self, lat, lon):
        if self.seoul_end_lat > lat or lat > self.seoul_start_lat:
            return False
        if self.seoul_end_lon < lon or lon < self.seoul_start_lon:
            return False
        return True

In [5]:
class Patient:
    def age_category(ages):
        categories = []
        for age in ages:
            age = int(age[:-1])
            if age == 0: category = 0
            elif age == 100: category = 10
            else: category = age // 10
            categories.append(category)
        return categories
            
    def sex_category(sexes):
        categories = []
        for sex in sexes:
            if sex == 'male': categories.append(0)
            else: categories.append(1)
        return categories
        
    def infection_case_category(infection_cases, causes):
        categories = []
        for infection_case in infection_cases:
            if infection_case not in causes:
                infection_case = "community infection"
            category = causes.index(infection_case)
            categories.append(category)
        return categories
    
    def type_category(types, move_types):
        categories = []
        for purpose in types:
            category = move_types.index(purpose)
            categories.append(category)
        return categories
    
    def day_category(days):
        categories = []
        for day in days:
            day = datetime.strptime(day, "%Y-%m-%d")
            category = day.weekday()
            categories.append(category)
        return categories

In [6]:
class Tensor:
    def __init__(self, names, counts, move_types, infection_cases, input_route, input_dates):
        self.names = names
        self.counts = counts
        self.move_types = move_types
        self.causes = infection_cases
        self.route = input_route
        self.dates = input_dates
        self.size_of_image = 256
        self.clear()
        
    def clear(self):
        all_count = sum(count for count in self.counts)
        # number of dates, number of features, row, col 
        first_day = datetime.strptime(self.dates[0], "%Y-%m-%d")
        last_day = datetime.strptime(self.dates[len(self.dates) - 1], "%Y-%m-%d")
        delta = last_day - first_day
        self.tensor = np.zeros((delta.days + 1, all_count, self.size_of_image, self.size_of_image), dtype=np.float)
            
    def age_category(self, ages):
        categories = []
        for age in ages:
            age = int(age[:-1])
            if age == 0: category = 0
            elif age == 100: category = 10
            else: category = age // 10
            categories.append(category)
        return categories
            
    def sex_category(self, sexes):
        categories = []
        for sex in sexes:
            if sex == 'male': categories.append(0)
            else: categories.append(1)
        return categories
        
    def infection_case_category(self, infection_cases):
        categories = []
        for infection_case in infection_cases:
            if infection_case not in self.causes:
                infection_case = "community infection"
            category = self.causes.index(infection_case)
            categories.append(category)
        return categories
    
    def type_category(self, types):
        categories = []
        for purpose in types:
            category = self.move_types.index(purpose)
            categories.append(category)
        return categories
    
    def day_category(self, days):
        categories = []
        for day in days:
            day = datetime.strptime(day, "%Y-%m-%d")
            category = day.weekday()
            categories.append(category)
        return categories
    
    def put_all_categories(self, order, ages, sexes, infection_cases, types, days, lats, lons):
        converter = GridMap()
        length = len(ages)
        
        ages_categories = self.age_category(ages)
        sexes_categories = self.sex_category(sexes)
        infection_case_categories = self.infection_case_category(infection_cases)
        type_categories = self.type_category(types)
        days_categories = self.day_category(days)
        
        for i in range(length):
            x, y = converter.toGrid(lats[i], lons[i])
            index = 0
            # age
            if ages_categories[i] != -1:
                age = ages_categories[i]
                self.tensor[order][age][x][y] += 10
                index += self.counts[0]
            # sex
            if sexes_categories[i] != -1:
                sex = index + sexes_categories[i]
                self.tensor[order][sex][x][y] += 10
                index += self.counts[1]
            # infection_case
            if infection_case_categories[i] != -1:
                infection_case = index + infection_case_categories[i]
                self.tensor[order][infection_case][x][y] += 10
                index += self.counts[2]
            # type
            if type_categories[i] != -1:
                visit = index + type_categories[i]
                self.tensor[order][visit][x][y] += 10
                index += self.counts[3]
            # day
            if days_categories[i] != -1:
                day = index + days_categories[i]
                self.tensor[order][day][x][y] += 10
                
    def generate_image(self):
        self.clear()
        first_day = datetime.strptime(self.dates[0], "%Y-%m-%d")
        last_day = datetime.strptime(self.dates[len(self.dates) - 1], "%Y-%m-%d")
        delta = last_day - first_day
        
        for i in range(delta.days + 1):
            today = (first_day + timedelta(days=i)).strftime("%Y-%m-%d")
            today = self.route.loc[self.route['date'] == today]
            
            # features in routes
            ages = today['age'].tolist()
            sexes = today['sex'].tolist()
            infection_cases = today['infection_case'].tolist()
            types = today['type'].tolist()
            dates = today['date'].tolist()
            lats = today['latitude'].tolist()
            lons = today['longitude'].tolist()
            
            # put in tensor
            self.put_all_categories(i, ages, sexes, infection_cases, types, dates, lats, lons)
        
        return self.tensor
    
    def get_tensor(self, start_day, duration):
        first_day = datetime.strptime(self.dates[0], "%Y-%m-%d")
        start_day = datetime.strptime(start_day, "%Y-%m-%d")
        delta = start_day - first_day
        day_index = delta.days
        
        origin = self.tensor[day_index:day_index+duration]
        return origin

In [7]:
# names, counts, move_types, input_route, input_dates
info = Route() 
route, dates = info.get_route_info()
club_route, club_dates = info.get_club_route_info()

names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 21, 7]
move_types = ['karaoke', 'gas_station', 'gym', 'bakery', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'pharmacy', 'airport']
infection_cases = info.causes
print(infection_cases)

['community infection', 'etc', 'contact with patient', 'overseas inflow']


In [8]:
def append3(array, row, col, value, size):
    print("append3 method in...")
    print("이미지 구조: {}".format(array.shape))
    stride = int((size - 1) / 2)
    ratio = 1 / (stride + 1)

    c = 1
    for i in range(stride, 0, -1):
        if row - i >=0 and row - i < array.shape[0] and col-i>=0 and col-i<array.shape[1]:
            array[row - i][col-i] += (c * ratio * value)

        for j in range(col-i + 1, col+i):
            if row-i >= 0 and j >= 0 and row-i<array.shape[0] and j<array.shape[1]:
                array[row-i][j] += (c * ratio * value)
            if j >= 0 and j < array.shape[1] and row+i<array.shape[0]:
                array[row+i][j] += (c * ratio * value)

        if row - i >= 0 and row-i<array.shape[0] and col+i <array.shape[1]:
            array[row - i][col+i] += (c * ratio * value)

        for j in range(row+i, row-i, -1):
            if col + i >= 0 and col+i<array.shape[1] and j >=0 and j<array.shape[1]:
                array[j][col + i] += (c * ratio * value)
            if col - i >= 0 and col-i<array.shape[1] and j<array.shape[1] and j>=0:
                array[j][col - i] += (c * ratio * value)
        c += 1
    
    array[row][col] = value
    
    return array

def save_images(final_tensors):
    day = 0
    print("저장할 이미지 4차원 배열:", final_tensors.shape) # (13, 45, 256, 256)
    for i in range(final_tensors.shape[0]):
        image_to_save1 = final_tensors[i]
        print("day {}에서 final_tensors의 구조: {}".format(day, image_to_save1.shape)) # (45, 256, 256)
        feature = 0
        for j in range(image_to_save1.shape[0]):
            print("{}번째 피처에서의 이미지 구조: {}".format(feature, image_to_save1.shape))
            image_to_save2 = image_to_save1[j:j+1,0:256,0:256]
            print("kernel 씌우기 전 이미지 구조1: {}".format(image_to_save2.shape))
            image_to_save2 = image_to_save2[0, :, :]
            print("kernel 씌우기 전 이미지 구조2: {}".format(image_to_save2.shape))
            image_to_save2 = overlay_kernel(image_to_save2)
            Imageee.generate_imageee('covid_images/test27/%dday%dfeature.png' % (day, feature), image_to_save2)
            feature += 1
        day += 1

def overlay_kernel(image_array):
    print("overlay_kerel method in...")
    print("이미지 구조: {}".format(image_array.shape))
    
    new_image = np.zeros((image_array.shape[0],image_array.shape[1]))
    for row in range(image_array.shape[0]):
        for col in range(image_array.shape[1]):
            if image_array[row][col] == 0: continue
            new_image += append3(np.zeros((image_array.shape[0],image_array.shape[1])), row, col, image_array[row][col], 13)
    image_array = new_image
        
    return new_image
    

def get_patient_tensors(route, patient_id, all_counts):
    patient = route.loc[route['patient_id']==patient_id]
    patient_dates = patient['date'].tolist()
    patient_dates.sort()
    
    max_value = 0

    # 날짜별 정렬
#     print("%s 환자 tensor get..." % (str(patient_id)))
    first_day = datetime.strptime(patient_dates[0], "%Y-%m-%d")
    last_day = datetime.strptime(patient_dates[len(patient_dates) - 1], "%Y-%m-%d")
    delta = last_day - first_day

    # 순서대로 날짜 가져옴
    patient_tensors = []
    for i in range(delta.days + 1):
        channels = []
        today = (first_day + timedelta(days=i)).strftime("%Y-%m-%d")
        today = patient.loc[patient['date'] == today]
        day_tensor, max_value_p = get_day_tensor(today, all_counts)    
        if max_value < max_value_p: max_value = max_value_p
        patient_tensors.append(day_tensor)
    
    patient_tensors = np.asarray(patient_tensors)
    
    return patient_tensors, first_day, delta.days + 1, last_day, max_value
    
def propagate_patient_tensors(patient_tensors):
    #print(patient_tensors[0].shape) #(45,256,256)
    total_days = patient_tensors.shape[0]
    if total_days <= 1: return patient_tensors
    
    second = total_days - 1
    first = second - 1
    
    for i in range(2 * total_days - 3):
        patient_tensors[second] = propagate_two_blocks(patient_tensors[first], patient_tensors[second])
        if second - first == 2: second -=1
        else: first -= 1
            
    return patient_tensors

def propagate_two_blocks(first_block, second_block):
#     print("propagate_two_blocks method in....") 
#     print("structure of first block: {}".format(first_block.shape)) # (45, 256, 256)
    channel_count = first_block.shape[0]
    for i in range(channel_count):
        second_block[i, 0:256, 0:256] += first_block[i, 0:256, 0:256]
    return second_block

def propagate_patienta(visits):
    second = len(visits) - 1
    first = second - 1
    
    for i in range(2 * days - 3):
        visits[second] = Imageee.propagate_2(visits[first], visits[second])
        if second - first == 2: second -=1
        else: first -= 1
            
    return visits

def get_day_tensor(today, all_counts):
    max_value = 0
    # 가져온 날 features
    p_ages = today['age'].tolist()
    p_sexes = today['sex'].tolist()
    p_infection_cases = today['infection_case'].tolist()
    p_types = today['type'].tolist()
    p_dates = today['date'].tolist()
    p_lats = today['latitude'].tolist()
    p_lons = today['longitude'].tolist()

    ages_categories = Patient.age_category(p_ages)
    sexes_categories = Patient.sex_category(p_sexes)
    infection_case_categories = Patient.infection_case_category(p_infection_cases, infection_cases)
    type_categories = Patient.type_category(p_types, move_types)
    days_categories = Patient.day_category(p_dates)

    converter = GridMap()
    length = len(p_ages)

    day_tensor = np.zeros((all_counts, 256, 256))

    for i in range(length):
        x, y = converter.toGrid(p_lats[i], p_lons[i])
        index = 0
        # age
        if ages_categories[i] != -1:
            age = ages_categories[i]
            day_tensor[age][x][y] += 60
            if max_value < day_tensor[age][x][y]: max_value = day_tensor[age][x][y]
            index += counts[0]
        # sex
        if sexes_categories[i] != -1:
            sex = index + sexes_categories[i]
            day_tensor[sex][x][y] += 60
            if max_value < day_tensor[sex][x][y]: max_value = day_tensor[age][x][y]
            index += counts[1]
        # infection_case
        if infection_case_categories[i] != -1:
            infection_case = index + infection_case_categories[i]
            day_tensor[infection_case][x][y] += 60
            if max_value < day_tensor[infection_case][x][y]: max_value = day_tensor[age][x][y]
            index += counts[2]
        # type
        if type_categories[i] != -1:
            visit = index + type_categories[i]
            day_tensor[visit][x][y] += 60
            if max_value < day_tensor[visit][x][y]: max_value = day_tensor[age][x][y]
            index += counts[3]
        # day
        if days_categories[i] != -1:
            day = index + days_categories[i]
            day_tensor[day][x][y] += 60
            if max_value < day_tensor[day][x][y]: max_value = day_tensor[age][x][y]
    return day_tensor, max_value / 60

In [11]:
# propagate 실제 환자 데이터 가져와서 적용
# 전체 루트
info = Route() 
route, dates = info.get_route_info()

# 피처 조건
names = ['age', 'sex', 'infection_case', 'type', 'date']
counts = [11, 2, 4, 21, 7]
move_types = ['karaoke', 'gas_station', 'gym', 'bakery', 'pc_cafe',
              'beauty_salon', 'school', 'church', 'bank', 'cafe',
              'bar', 'post_office', 'real_estate_agency', 'lodging',
              'public_transportation', 'restaurant', 'etc', 'store',
              'hospital', 'pharmacy', 'airport']
infection_cases = info.causes
all_counts = sum(feature for feature in counts)

# 환자별 정렬
patient_ids = route['patient_id']
patient_ids = patient_ids.drop_duplicates(keep='last')
patient_ids = patient_ids.tolist()
patient_ids.sort()

# 모든 환자 정보 가져옴
all_patients = []
all_first_dates = []
count = 0
count_p = 0
max_value = 0
for patient_id in patient_ids:
#     if count < 200:
#         count += 1
#         continue
#     if count >= 10: break
    if count % 10 == 0: print("%d명 진행중..." % count)
    patient_tensors, first_day, duration, last_day, max_value_p = get_patient_tensors(route, patient_id, all_counts)
    if max_value < max_value_p: max_value = max_value_p
    print(patient_id, first_day, duration, last_day) #test
    patient_tensors = propagate_patient_tensors(patient_tensors)
    all_first_dates.append([first_day, duration, count, patient_id, last_day])
    all_patients.append(patient_tensors)
    count += 1
    count_p += 1
    
print("max value in this case: {}".format(max_value))

# test(7명) - 각 일자별로 환자 들어오는지 확인(propagate 없음)
# for i in range(len(all_first_dates)):
#     print(all_first_dates[i])
#     duration = all_first_dates[i][1]
#     patient_tensor = all_patients[i]
#     for j in range(duration):
#         day_tensor = patient_tensor[j]
#         for k in range(day_tensor.shape[0]): # 채널
#             patient_channel = day_tensor[k]
#             print(patient_channel.shape)
#             Imageee.generate_imageee('covid_images/test11/%s_%d_%d.png' % (all_first_dates[i][3], j, k), patient_channel)
        
# test(7명) - 각 일자별로 환자 들어오는지 확인(propagate 있음)
# for i in range(len(all_first_dates)):
#     print(all_first_dates[i])
#     duration = all_first_dates[i][1]
#     patient_tensor = all_patients[i]
#     for j in range(duration):
#         day_tensor = patient_tensor[j]
#         for k in range(day_tensor.shape[0]): # 채널
#             patient_channel = day_tensor[k]
#             print(patient_channel.shape)
#             Imageee.generate_imageee('covid_images/test12/%s_%d_%d.png' % (all_first_dates[i][3], j, k), patient_channel)
    
    
# 같은날 환자별로 모음
# 활동 일수가 0인 환자부터 제외
# print(all_first_dates)
all_first_dates = [elem for elem in all_first_dates if elem[1] != 0]
# print(all_first_dates)
# print()
all_first_dates = sorted(all_first_dates,key=lambda l:l[0])
# print(all_first_dates)

first_day = all_first_dates[0][0]
last_day = np.max(all_first_dates, axis=0)[4]
today = first_day
print(first_day, last_day)
print()

final_tensors = []
patients_by_date = []
while(True):
    print(today)
    # today가 활동 시작인 확진자 추가
    patients_by_date.extend([elem for elem in all_first_dates if elem[0] == today])
    print(patients_by_date)
    
    ## to_do 
    day_accumulated = np.zeros((all_counts, 256, 256))
    for x in patients_by_date: # 같은 날짜에 활동중인 사람들
        patient_index_in_array = x[2] # 전체 배열 중에서 환자 인덱스
        patient_index = all_patients[x[2]].shape[0] - x[1] # 환자 3d 배열 중 today에 해당하는 인덱스
        print("patient_index: {}, {}".format(x[3], patient_index))
        day_accumulated = propagate_two_blocks(all_patients[x[2]][patient_index], day_accumulated)
    final_tensors.append(day_accumulated)
    
    # 활동 일수 끝난 환자 삭제
    for x in patients_by_date: x[1] -= 1
    patients_by_date = [elem for elem in patients_by_date if elem[1] != 0]
    
    # while break 조건
    if today == last_day: break
    today += timedelta(days=1)

final_tensors = np.asarray(final_tensors)
# save_images(final_tensors) # + KDE도 같이
# print("start to save image...")

final_tensors.shape

0명 진행중...
1000000001 2020-01-24 00:00:00 1 2020-01-24 00:00:00
1000000002 2020-01-25 00:00:00 6 2020-01-30 00:00:00
1000000003 2020-01-26 00:00:00 1 2020-01-26 00:00:00
1000000004 2020-01-30 00:00:00 1 2020-01-30 00:00:00
1000000005 2020-01-31 00:00:00 1 2020-01-31 00:00:00
1000000008 2020-01-31 00:00:00 1 2020-01-31 00:00:00
1000000009 2020-01-31 00:00:00 6 2020-02-05 00:00:00
1000000010 2020-01-29 00:00:00 8 2020-02-05 00:00:00
1000000011 2020-02-02 00:00:00 5 2020-02-06 00:00:00
1000000013 2020-02-05 00:00:00 12 2020-02-16 00:00:00
max value in this case: 2.0
2020-01-24 00:00:00 2020-02-16 00:00:00

2020-01-24 00:00:00
[[datetime.datetime(2020, 1, 24, 0, 0), 1, 0, 1000000001, datetime.datetime(2020, 1, 24, 0, 0)]]
patient_index: 1000000001, 0
2020-01-25 00:00:00
[[datetime.datetime(2020, 1, 25, 0, 0), 6, 1, 1000000002, datetime.datetime(2020, 1, 30, 0, 0)]]
patient_index: 1000000002, 0
2020-01-26 00:00:00
[[datetime.datetime(2020, 1, 25, 0, 0), 5, 1, 1000000002, datetime.datetime(20

(24, 45, 256, 256)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import keras.backend.tensorflow_backend as K
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pylab as plt

In [ ]:
rs = 256
n_step = 3
n_test = 1

In [ ]:
# We create a layer which take as input movies of shape
# (n_frames, width, height, channels) and returns a movie
# of identical shape.
def get_model(channel):
    with K.tf_ops.device('/device:GPU:0'):
        seq = Sequential()
        seq.add(ConvLSTM2D(filters=rs, kernel_size=(3, 3), data_format='channels_first',
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=rs, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=rs, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(ConvLSTM2D(filters=rs, kernel_size=(3, 3),
                           padding='same', return_sequences=True))
        seq.add(BatchNormalization())

        seq.add(Conv3D(filters=channel, kernel_size=(3, 3, 3),
                       activation='sigmoid',
                       padding='same', data_format='channels_first'))
        
        seq.compile(loss='mean_squared_error', optimizer='rmsprop')
#         seq.compile(loss='binary_crossentropy', optimizer='adadelta')
    return seq

In [ ]:
image = final_tensors

n = image.shape[0]
channel = image.shape[1]

n_test = n_step * 2
n_train = n - n_test

train = image

X_train = []
y_train = []
for i in range(n_step,n_train-n_step):
    X_train.append(train[i-n_step:i, :,:])
    y_train.append(train[i:i+n_step, :,:])
X_train, y_train = np.array(X_train), np.array(y_train)

seq = get_model(channel)
seq.fit(X_train, y_train, epochs = 200, batch_size = 32)
seq.save('model/2020-05-16-train_seq_image.h5')

In [ ]:
---

In [ ]:
predictions = []

for j, image in enumerate(club_images):
    n = image.shape[0]
    channel = image.shape[1]

    n_test = n_step * 2
    n_train = n - n_test

    test = image
    
    X_test = []
    y_test = []
    for i in range(n_step,n_train-n_step):
        X_test.append(test[i-n_step:i, :,:])
        y_test.append(test[i:i+n_step, :,:])
    X_test, y_test = np.array(X_test), np.array(y_test)
    print(X_test.shape, y_test.shape)
    
    seq = load_model('model/train_seq_image%d_2.h5' % i)
    #prediction
    pred = seq.predict(X_test)
    predictions.append(pred)
    print(pred.shape)
    pred=pred.reshape(n_step, channel, rs, rs)

    real_vv=[]
    pre_vv=[]
    for t in range(n_step):
        tmp=y_test[t].reshape((channel, rs,rs))
        xx=np.argwhere(False==np.isnan(tmp))[:,0]
        yy=np.argwhere(False==np.isnan(tmp))[:,1]
        real_vv.append(tmp[xx,yy])
        pre_vv.append(pred[t,xx,yy])

    real_vv=np.array(real_vv)
    pre_vv=np.array(pre_vv)
    print(real_vv.shape)
    print(pre_vv.shape)

    for i in range(len(real_vv)):
        rmse=np.sqrt(mean_squared_error(real_vv[i],pre_vv[i]))
        print(i+1," : ",'Test RMSE: %.3f'%rmse)